# Notebook to attempt a simulation of generated microstructures

In [ ]:
import datetime
import logging
import dataclasses
import os
import sys
import pickle
from typing import Optional, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np

from spins import goos
from spins.goos_sim import maxwell


In [ ]:
# set-up with saving folder, and optimization plan
out_folder_name = "grating_full_opt_gpu_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #The folder will be saved in My Drive of the google drive. 
folder_plt = out_folder_name #Plotting folder is separately here, in case one wishes to plot from another folder. 
out_folder = os.path.join(os.getcwd(),out_folder_name)
if (not os.path.exists(out_folder)):
  os.makedirs(out_folder)

goos.util.setup_logging(out_folder)
plan = goos.OptimizationPlan(save_path = out_folder)

In [ ]:
out_folder

In [ ]:
# Major params
theta = np.radians(60)
L = 900 #nm
h = 325

In [ ]:
P = L / np.sin(theta)
# P = np.ceil(P).astype('int')
P

In [ ]:
p = P / 256
p

In [ ]:
H = 325 + 325 + 325 + 25
H

In [ ]:
# set - up variables needed for grating. 

@dataclasses.dataclass
class Options:
    """Maintains list of options for the optimization.

    Attributes:
        coupler_len: Length of the grating coupler.
        wg_width: Width of the grating coupler. Only relevant for GDS file
            generation.
        wg_len: Length of the waveguide to which the grating coupler couples.
        wg_thickness: Thickness of the waveguide.
        etch_frac: Etch fraction of the grating.
        min_features: Minimum feature sizes.
        box_size: Thickness of the buried oxide layer.
        source_angle_deg: Angle of the Gaussian beam in degrees relative to
            the normal.

        buffer_len: Additional distance to add to the top and bottom of the
            simulation for simulation accuracy.

        eps_bg: Refractive index of the background.
        eps_fg: Refraction index of the waveguide/grating.

        beam_dist: Distance of the Gaussian beam from the grating.
        beam_width: Diameter of the Gaussian beam.
        beam_extents: Length of the Gaussian beam to use in the simulation.

        wlen: Wavelength to simulate at.
        dx: Grid spacing to use in the simulation.
        pixel_size: Pixel size of the continuous grating coupler
            parametrization.
    """
    # We want a 2D sim
    coupler_len: float = P
    grating_width: float = P
    grating_len: float = 1
    grating_thickness: float = h  # nm
    box_size: float = h
    source_angle_deg: float = 0
    buffer_len: float = 25
    eps_bg: float = 1.45
    eps_grating: float = 3.62
    eps_air: float = 1.00 
    beam_dist: float = h + 25
    beam_width: float = P
    beam_extents: float = P
    wlen: float = L
    dx: float = 1
    pixel_size: float = 1

    etch_frac: float = 1.0
    min_features: float = 4

params = Options()

In [ ]:
?goos.Cuboid

In [ ]:
#set-up background shapes
with plan:
    air = goos.Cuboid(
        pos=goos.Constant([h, 0, 0]),
        extents=goos.Constant([h, P, 1]),
        material=goos.material.Material(index=params.eps_air)
    )

    substrate = goos.Cuboid(
            pos=goos.Constant([-h, 0, 0]),
            extents=goos.Constant([h, P, 1]),
            material=goos.material.Material(index=params.eps_bg)
    )

    structure = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    blocks = []
    for i, material in enumerate(structure):
        if material == 1:
            ri = params.eps_grating
        else:
            ri = params.eps_air

        x_pos = -0.5 * (P - p - 2*i*p)
        
        block = goos.Cuboid(
            pos=goos.Constant([0, x_pos, 0]),
            extents=goos.Constant([h, p, 1]),
            material=goos.material.Material(index=ri)
        )
        blocks.append(block)
    
    eps_structures = goos.GroupShape(blocks + [substrate, air])
    

In [ ]:
'''
Visualize the constant background structures we just defined.
'''
with plan:
  eps_rendered = maxwell.RenderShape(
            eps_structures,
            region=goos.Box3d(center=[0, 0, 0], extents=[P, P, 0]),
            mesh=maxwell.UniformMesh(dx=p/8),
            wavelength=900,
        )
  
  goos.util.visualize_eps(eps_rendered.get().array[2])

In [ ]:
# set-up design area and finish eps we need. 

# with plan:        

#     def initializer(size):
#         return np.random.random(size)

#     # Continuous optimization.
#     var, design_cont = goos.pixelated_cont_shape(
#         initializer=initializer,
#         pos=goos.Constant([
#             params.coupler_len / 2, 0,
#             params.wg_thickness / 2 * (1 - params.etch_frac)
#         ]),
#         extents=[
#             params.coupler_len, params.wg_width,
#             params.wg_thickness * params.etch_frac
#         ],
#         material=goos.material.Material(index=params.eps_bg),
#         material2=goos.material.Material(index=params.eps_wg),
#         pixel_size=[
#             params.pixel_size, params.wg_width, params.wg_thickness
#         ])

#     eps_cont = goos.GroupShape([substrate, waveguide, wg_bottom, design_cont])

In [ ]:
with plan:
    # Define wavelength and solver.
    my_wavelength = 900
    sim_z_extent = 1
    solver_info = maxwell.MaxwellSolver(solver="maxwell_cg",
                                        err_thresh=1e-2)
    pml_thickness = [0, 0, 400, 400, 400, 400]

    # Define simulation space.
    my_simulation_space = maxwell.SimulationSpace(
        mesh=maxwell.UniformMesh(dx=p/8),
        sim_region=goos.Box3d(
            center=[0, 0, 0],
            extents=[P, P, sim_z_extent],
        ),
        pml_thickness=pml_thickness,
    )
  
    # Define a waveguide mode source.
    my_sources = [maxwell.WaveguideModeSource(center=[0, -P/2, 0],
                                              extents=[0, P, 1],
                                              normal=[0, 1, 0],
                                              mode_num=0,
                                              power=1)]

      
    # Define simulation outputs.
    my_outputs=[ maxwell.Epsilon(name="eps"),
                 maxwell.ElectricField(name="field")]

    # Setup the simulation object.
    sim = maxwell.fdfd_simulation(
        name="sim_cont",
        wavelength=L,
        background=goos.material.Material(index=1.0),
        eps=eps_structures,
        simulation_space = my_simulation_space,
        solver_info = solver_info,
        sources = my_sources,
        outputs= my_outputs
    )

In [ ]:
# Set-up continuous optimization objective function with eps.
# with plan:

#     sim_left_x = -params.wg_len
#     sim_right_x = params.coupler_len + params.buffer_len
#     pml_thick = params.dx * 10
#     sim_z_center = (params.wg_thickness / 2 + params.beam_dist -
#                     params.box_size) / 2
#     sim_z_extent = (params.wg_thickness + params.beam_dist + params.box_size +
#                     2000 + pml_thick * 2)

#     sources=[
#             maxwell.GaussianSource(
#                 w0=params.beam_width / 2,
#                 center=[
#                     params.coupler_len / 2, 0,
#                     params.wg_thickness / 2 + params.beam_dist
#                 ],
#                 extents=[params.beam_extents, 0, 0],
#                 normal=[0, 0, -1],
#                 power=1,
#                 theta=np.deg2rad(params.source_angle_deg),
#                 psi=np.pi / 2,
#                 polarization_angle=0,
#                 normalize_by_sim=True)
#         ]
#     outputs=[
#       maxwell.Epsilon(name="eps"),
#       maxwell.ElectricField(name="field"),
#       maxwell.WaveguideModeOverlap(name="overlap",
#                                    center=[-params.wg_len / 2, 0, 0],
#                                    extents=[0, 1000, 2000],
#                                    normal=[-1, 0, 0],
#                                    mode_num=0,
#                                    power=1),
#      ]
#     simulation_space=maxwell.SimulationSpace(
#         mesh=maxwell.UniformMesh(dx=params.dx),
#         sim_region=goos.Box3d(
#             center=[(sim_left_x + sim_right_x) / 2, 0, sim_z_center],
#             extents=[sim_right_x - sim_left_x, 0, sim_z_extent],
#             ),
#         pml_thickness=[pml_thick, pml_thick, 0, 0, pml_thick, pml_thick])
    
#     sim_cont = maxwell.fdfd_simulation(
#         name="sim_{}".format("cont"),
#         simulation_space = simulation_space,
#         wavelength=params.wlen,
#         sources = sources,
#         eps=eps_cont,
#         solver="maxwell_cg",
#         outputs=outputs,
#         background=goos.material.Material(index=1.444),

#     )



In [ ]:
# obj_c = (1 - goos.abs(sim_cont["overlap"]))**2 #elaborate how simple. It makes difference. This from our experience is the best. Try your options! 
# obj_c = goos.rename(obj_c, name="obj_{}".format("cont"))
    

In [ ]:
# # set-up continuous optimization with scipy
# with plan:
#     cont_max_iter = 20
#     goos.opt.scipy_minimize(
#     obj_c,
#     "L-BFGS-B",
#     monitor_list=[sim_cont["eps"], sim_cont["field"], sim_cont["overlap"], obj_c],
#     max_iters=cont_max_iter,
#     name="opt_cont")

#     # Prevent optimization from optimizing over continuous variable.
#     var.freeze()


In [ ]:
# # set-up discrete optimization with scipy
# with plan:
#     goos.opt.scipy_minimize(
#         obj_d,
#         "L-BFGS-B",
#         monitor_list=[sim_disc["eps"], sim_disc["field"], sim_disc["overlap"], obj_d],
#         max_iters=20,
#         name="opt_disc",
#         ftol=1e-8)

In [ ]:
# run the optimization
with plan:
    plan.save()
    plan.run()

In [ ]:
#visualizing the initial structure permittivity and the field.  
with open(os.path.join(folder_plt, "step{}.pkl".format(1)), "rb") as fp:
  data = pickle.load(fp)
  
  plt.figure(figsize=(10,12))
  plt.imshow(
      np.rot90(np.abs(data["monitor_data"]["sim_cont.eps"][0].squeeze()),1,(0,1)))
  plt.axis("off")
  plt.tight_layout()
  plt.figure(figsize=(10,12))
  plt.imshow(
      np.rot90(np.abs(
        data["monitor_data"]["sim_cont.field"][1].squeeze()),1,(0,1)))
  plt.axis("off")
  plt.tight_layout()
  plt.show()
  print("Overlap transmission value is " + str(np.abs(data["monitor_data"]["sim_cont.overlap"])**2))


In [ ]:
#visualizing end of continous optimization
with open(os.path.join(folder_plt, "step{}.pkl".format(cont_max_iter)), "rb") as fp:
  data = pickle.load(fp)
  
  plt.figure(figsize=(10,12))
  plt.imshow(
      np.rot90(np.abs(data["monitor_data"]["sim_cont.eps"][0].squeeze()),1,(0,1)))
  plt.axis("off")
  plt.tight_layout()
  plt.figure(figsize=(10,12))
  plt.imshow(
      np.rot90(np.abs(
        data["monitor_data"]["sim_cont.field"][1].squeeze()),1,(0,1)))
  plt.axis("off")
  plt.tight_layout()
  plt.show()
  print("Overlap transmission value is " + str(np.abs(data["monitor_data"]["sim_cont.overlap"])**2))

In [ ]:
#visualizing the structure and the field at the end of the discretization
with open(os.path.join(folder_plt, "step{}.pkl".format(cont_max_iter+1)), "rb") as fp:
  data = pickle.load(fp)
  
  plt.figure(figsize=(10,12))
  plt.imshow(
      np.rot90(np.abs(data["monitor_data"]["sim_disc.eps"][0].squeeze()),1,(0,1)))
  plt.axis("off")
  plt.tight_layout()
  plt.figure(figsize=(10,12))
  plt.imshow(
      np.rot90(np.abs(
        data["monitor_data"]["sim_disc.field"][1].squeeze()),1,(0,1)))
  plt.axis("off")
  plt.tight_layout()
  plt.show()
  print("Overlap transmission value is " + str(np.abs(data["monitor_data"]["sim_disc.overlap"])**2))

In [ ]:
#visualizing the structure and the field at the end of the optimization
step = goos.util.get_latest_log_step(folder_plt)
with open(os.path.join(folder_plt, "step{}.pkl".format(step)), "rb") as fp:
  data = pickle.load(fp)

plt.figure(figsize=(10,12))
plt.imshow(
    np.rot90(np.abs(data["monitor_data"]["sim_disc.eps"][0].squeeze()),1,(0,1)))
plt.axis("off")
plt.tight_layout()

plt.figure(figsize=(10,12))
plt.imshow(
    np.rot90(np.abs(
      data["monitor_data"]["sim_disc.field"][1].squeeze()),1,(0,1)))
plt.axis("off")
plt.tight_layout()

plt.show()
print("Overlap transmission value is " + str(np.abs(data["monitor_data"]["sim_disc.overlap"])**2))


    

In [ ]:
#Reading all pkl files in the saving folder to see optimization trajectory over iterations. 
disc_last_step = goos.util.get_latest_log_step(folder_plt)
transmission = []
for step in range(1, cont_max_iter+1):
  with open(os.path.join(folder_plt, "step{}.pkl".format(step)), "rb") as fp:
    data = pickle.load(fp)
    transmission.append(np.abs(data["monitor_data"]["sim_cont.overlap"])**2)
for step in range(cont_max_iter+1, int(disc_last_step)+1):
  with open(os.path.join(folder_plt, "step{}.pkl".format(step)), "rb") as fp:
    data = pickle.load(fp)
    transmission.append(np.abs(data["monitor_data"]["sim_disc.overlap"])**2)


In [ ]:
#plotting the overlap values for the all pkl files in the saving folder to see optimization trajectory over iterations. 
plt.figure(figsize=(12,6))
plt.plot(range(1,int(disc_last_step)+1),transmission)
plt.xlabel("Iteration")
plt.ylabel("Transmission")
plt.tight_layout()
